In [2]:
import glob
import os
import shutil

import numpy as np
import cv2
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline
from moviepy.editor import VideoFileClip, ImageSequenceClip
import helper

In [9]:
from importlib import reload
reload(helper)

cal_path = 'camera_cal/'
output_images = 'output_images/'
path = 'test_images/'

def pipeline(img, mtx, dist, roi_vertices, M, M_inv, left_line, right_line, bad_frames):
    # Undistort image
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    imshape = undist.shape

    # Apply thresholds to image and combine them
    # Choose a Sobel kernel size
    ksize = 5
    
    # Convert to HLS color space and separate the S channel
    hls = cv2.cvtColor(undist, cv2.COLOR_RGB2HLS)
    h_channel = hls[...,0]
    h_channel = helper.clahe(h_channel, 4, (5, 5))
    l_channel = hls[...,1]
    l_channel = helper.clahe(l_channel, 6, (15, 15))
    s_channel = hls[...,2]
    s_channel = helper.clahe(s_channel, 4, (5, 5))

    # Apply thresholds to image and combine them
    # Choose a Sobel kernel size
    ksize = 7
    close_kernel = np.ones((5,5),np.uint8)

    gradx = helper.abs_sobel_thresh(s_channel, orient='x', kernel=ksize,
                                    thresh=(50, 200))
    gradx = cv2.morphologyEx(gradx, cv2.MORPH_CLOSE, kernel=close_kernel)
    grady = helper.abs_sobel_thresh(s_channel, orient='y', kernel=ksize,
                                    thresh=(70, 200))
    mag_binary = helper.mag_thresh(s_channel, kernel=ksize, thresh=(30, 200))
    gradients = np.zeros_like(gradx)
    gradients[((gradx == 1) | (grady == 1)) & (mag_binary == 1)] = 1
    
    yellow_min = np.array([15, 60, 60], np.uint8)
    yellow_max = np.array([90, 200, 255], np.uint8)
    yellow_mask = cv2.inRange(hls, yellow_min, yellow_max)
    
    white_min = np.array([0, 200, 0], np.uint8)
    white_max = np.array([255, 255, 255], np.uint8)
    white_mask = cv2.inRange(hls, white_min, white_max)

    binary_output = np.zeros_like(gradients)
    binary_output[((yellow_mask == 255) | (white_mask == 255))] = 1

    color_binary_h = helper.color_threshold(h_channel, thresh=(30, 80))
    color_binary_l = helper.color_threshold(l_channel, thresh=(170, 255))
    color_binary_s = helper.color_threshold(s_channel, thresh=(100, 255))

    color_combined = np.zeros_like(binary_output)
    color_combined[(color_binary_h == 1) | ((color_binary_s == 1) | (color_binary_l == 1))] = 1

    # Combine gradient and color thresholding
    mask = np.zeros_like(binary_output)
    mask[(binary_output == 1) & ((gradients == 1) | (color_combined == 1))] = 1
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel=close_kernel)

    roi = helper.region_of_interest(mask, [roi_vertices])
    warped = cv2.warpPerspective(roi, M, roi.shape[::-1])

    # Convolution over warped image to find hot pixels defining lane lines
    conv_kernel = np.ones((5,5),np.uint8)
    conv = helper.conv_img(warped, conv_kernel)
    
    if (not left_line.detected or not right_line.detected) and bad_frames > 2 or \
    left_line.best_fit is None or right_line.best_fit is None:
        # Calculate initial lane centroids
        initial_fit = True
        left_x_centroids, left_y_centroids, \
        right_x_centroids, right_y_centroids = helper.window_search(conv) 
    else:
        # Calculate lane centroids using previous ones as support
        left_x_centroids, left_y_centroids = helper.guided_window_search(conv, left_line)
        right_x_centroids, right_y_centroids = helper.guided_window_search(conv, right_line)

    
    # Calculate base position and update lines
    ym_per_pix = 30 / 720
    xm_per_pix = 3.7 / 790

    halfway = 3.7 / 2
    car_pos = conv.shape[1] / 2
    if left_line.best_fit is None or right_line.best_fit is None:
        left_base_pos = None
        right_base_pos = None
        line_base_pos = 0
    else:
        left_pos = np.polyval(left_line.best_fit, conv.shape[1])
        right_pos = np.polyval(right_line.best_fit, conv.shape[1])
        lane_px = abs(left_pos - right_pos)
        lane_midpoint = lane_px / 2 + left_pos
        line_base_pos = (car_pos - lane_midpoint) * xm_per_pix
        left_base_pos = halfway + line_base_pos
        right_base_pos = halfway - line_base_pos

    # Update lines
    left_line = helper.update_line(conv, left_line, left_x_centroids, \
                                   left_y_centroids, left_base_pos)

    right_line = helper.update_line(conv, right_line, right_x_centroids, \
                                    right_y_centroids, right_base_pos)
    
    # Unwarp image
    ploty = np.linspace(0, conv.shape[0]-1, conv.shape[0])
    left_fitx = np.polyval(left_line.best_fit, ploty)
    right_fitx = np.polyval(right_line.best_fit, ploty)
    left_zipped = np.array(list(zip(left_fitx, ploty)), dtype=np.int32)
    right_zipped = np.array(list(zip(right_fitx, ploty)), dtype=np.int32)
    
    fitted = np.zeros_like(conv)
    fitted = np.dstack((fitted, fitted, fitted))
    cv2.fillPoly(fitted, [np.concatenate((left_zipped, right_zipped[::-1]))],
             (0,255, 0))
    
    # Uncomment to get a video side by side with the top-view of the lane
    # with the algorithm working
    
#     aux_fitted = np.zeros_like(fitted)
    
#     cv2.polylines(aux_fitted, [left_zipped], False, (0,0,255), 6, -1)
#     cv2.polylines(aux_fitted, [right_zipped], False, (0,0,255), 6, -1)
    
#     for values in left_line.recent_centroids_fitted:
#         for centroid in values:
#             cv2.circle(aux_fitted, (int(centroid[0]), int(centroid[1])),
#                        6, (0,255,0), -1)
#     for values in right_line.recent_centroids_fitted:
#         for centroid in values:
#             cv2.circle(aux_fitted, (int(centroid[0]), int(centroid[1])),
#                        6, (0,255,0), -1)
    
    unwarped = cv2.warpPerspective(fitted, M_inv, fitted.shape[1::-1])
    
    curverad = (left_line.radius_of_curvature + right_line.radius_of_curvature) / 2
    curverad = round(curverad, 2)
    cv2.putText(unwarped, 'Radius of curvature: ' + str(curverad) + 'm',
                (40, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255))

    line_base_pos = round(line_base_pos, 2)
    if line_base_pos < 0:
        cv2.putText(unwarped, 'Base position: ' + str(-line_base_pos) + 
                    'm left of center', (40, 60), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (255,255,255))
    else:
        cv2.putText(unwarped, 'Base position: ' + str(line_base_pos) + 
                    'm right of center', (40, 60), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (255,255,255))
    
    weighted = cv2.addWeighted(unwarped, 0.4, undist, 1., 0.)
    
    # Uncomment to get a video side by side with the top-view of the lane
    # with the algorithm working
    
#     aux_conv = np.dstack((conv, conv, conv))*128
#     aux_weighted = cv2.addWeighted(aux_fitted, 0.4, aux_conv, 1., 0.)
#     weighted = np.hstack((weighted, aux_weighted))
    
    return weighted, left_line, right_line

In [10]:
mtx, dist = helper.camera_calibration(cal_path, output_images)

bad_frames = 0
output_path = 'test_videos_output./output.mp4'
# input_video = './project_video.mp4'
input_video = 'test_videos/challenge.mp4'
clip = VideoFileClip(input_video)

AttributeError: module 'helper' has no attribute 'camera_calibration'

In [12]:
from importlib import reload
import helper
reload(helper)

left_line = helper.Line()
right_line = helper.Line()
out_frames = []
loops = 0

temp_dir = './temp/'
if not os.path.isdir(temp_dir):
    os.mkdir(temp_dir)

for frame in clip.iter_frames():

    new_frame, left_line, right_line = pipeline(frame, mtx, dist, roi_vertices,
                                                M, M_inv, left_line, right_line,
                                                bad_frames)
    if not left_line.detected or not right_line.detected:
        bad_frames += 1
    else:
        bad_frames = 0

    frame_name = temp_dir + 'temp_'+str(loops)+'.jpg'
    cv2.imwrite(frame_name, cv2.cvtColor(new_frame, cv2.COLOR_RGB2BGR))
    loops += 1
    out_frames.append(frame_name)

out_clip = ImageSequenceClip(out_frames, fps=clip.fps)
%time out_clip.write_videofile(output_path, audio=False)
# Comment the line below to keep the images generated by the pipeline
shutil.rmtree(temp_dir)

AttributeError: module 'helper' has no attribute 'Line'